In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.signal import resample
from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.backend import expand_dims
from sklearn.preprocessing import StandardScaler
from keras.layers import Input

#from keras.models import Model
#from keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape
#from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout

from utils.constants import NB_CLASSES_LIST, MAX_TIMESTEPS_LIST
from utils.keras_utils import train_model, evaluate_model, set_trainable
#from utils.layer_utils import AttentionLSTM

In [10]:
from keras.utils import multi_gpu_model
import os
# 使用第一张与第三张GPU卡
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [92]:
def acc_combo(y, y_pred):
    # 数值ID与行为编码的对应关系
    mapping = {0: 'A_0', 1: 'A_1', 2: 'A_2', 3: 'A_3', 
        4: 'D_4', 5: 'A_5', 6: 'B_1',7: 'B_5', 
        8: 'B_2', 9: 'B_3', 10: 'B_0', 11: 'A_6', 
        12: 'C_1', 13: 'C_3', 14: 'C_0', 15: 'B_6', 
        16: 'C_2', 17: 'C_5', 18: 'C_6'}
    # 将行为ID转为编码
    code_y, code_y_pred = mapping[y], mapping[y_pred]
    if code_y == code_y_pred: #编码完全相同得分1.0
        return 1.0
    elif code_y.split("_")[0] == code_y_pred.split("_")[0]: #编码仅字母部分相同得分1.0/7
        return 1.0/7
    elif code_y.split("_")[1] == code_y_pred.split("_")[1]: #编码仅数字部分相同得分1.0/3
        return 1.0/3
    else:
        return 0.0

def squeeze_excite_block(input):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
    Returns: a keras tensor
    '''
    filters = input.shape[-1] # channel_axis = -1 for TF

    se = GlobalAveragePooling1D()(input)
    se = Reshape((1, filters))(se)
    se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = multiply([input, se])
    return se

def get_dic(df,  main_col, fea_col, agg):
    dic = df.groupby(main_col)[fea_col].agg(agg).to_dict()
    fea_name = '_'.join([main_col, fea_col, agg])
    return fea_name, dic
    
def get_1st_order_xyz_features(df, fea_cols, main_col = 'fragment_id'): 
    df_fea           = pd.DataFrame()
    df_fea[main_col] = df[main_col].unique()
    ## count 特征 ##
    _, dic = get_dic(df, main_col, fea_cols[0], 'count') 
    df_fea['cnt']    = df_fea[main_col].map(dic).values
    
    ## 数值统计特征 ##
    for f in tqdm(fea_cols):
        for agg in ['min','max','mean','std','skew','median']:

            fea_name, dic       = get_dic(df, main_col, f, agg) 
            df_fea[fea_name]    = df_fea[main_col].map(dic).values
            
        df_fea['_'.join([main_col, f, 'gap'])]   = df_fea['_'.join([main_col, f, 'max'])] - df_fea['_'.join([main_col, f, 'min'])]
        df_fea['_'.join([main_col, f, 'skew2'])] = (df_fea['_'.join([main_col, f, 'mean'])] - df_fea['_'.join([main_col, f, 'median'])]) / df_fea['_'.join([main_col, f, 'std'])]
        
    return df_fea

In [25]:
train['behavior_id'].nunique()

19

In [ ]:

train = pd.read_csv('sensor_train.csv')
test = pd.read_csv('sensor_test.csv')
#train_fly = pd.read_csv('train_fly.csv')
#test_fly = pd.read_csv('test_fly.csv')
sub = pd.read_csv('sub.csv')
y = train.groupby('fragment_id')['behavior_id'].min()

train['mod'] = (train.acc_x ** 2 + train.acc_y ** 2 + train.acc_z ** 2) ** .5
train['modg'] = (train.acc_xg ** 2 + train.acc_yg ** 2 + train.acc_zg ** 2) ** .5
train['xy'] = (train.acc_x ** 2 + train.acc_y ** 2) ** .5
train['xy_g'] = (train.acc_xg ** 2 + train.acc_yg ** 2) ** .5
test['mod'] = (test.acc_x ** 2 + test.acc_y ** 2 + test.acc_z ** 2) ** .5
test['modg'] = (test.acc_xg ** 2 + test.acc_yg ** 2 + test.acc_zg ** 2) ** .5
test['xy'] = (test.acc_x ** 2 + test.acc_y ** 2) ** .5
test['xy_g'] = (test.acc_xg ** 2 + test.acc_yg ** 2) ** .5

#origin_fea_cols = ['acc_x','acc_y','acc_z','acc_xg','acc_yg','acc_zg','mod','modg','xy','xy_g']
#train_xyz_fea1 = get_1st_order_xyz_features(train,origin_fea_cols,main_col='fragment_id')
#test_xyz_fea1 = get_1st_order_xyz_features(test,origin_fea_cols,main_col='fragment_id')

#train = train.merge(train_xyz_fea1,how='left',on='fragment_id')
#test = test.merge(test_xyz_fea1,how='left',on='fragment_id')
feature_cols = [col for col in train.columns if col not in ['fragment_id','time_point','behavior_id']]
std_scaler = StandardScaler()
train[feature_cols] = std_scaler.fit_transform(train[feature_cols])
test[feature_cols] = std_scaler.transform(test[feature_cols])

feauture_len = len(feature_cols)

x = np.zeros((7292, 60, feauture_len, 1))
t = np.zeros((7500, 60, feauture_len, 1))
for i in tqdm(range(7292)):
    tmp = train[train.fragment_id == i][:60]
    x[i,:,:, 0] = resample(tmp.drop(['fragment_id', 'time_point', 'behavior_id'],
                                    axis=1), 60, np.array(tmp.time_point))[0]
for i in tqdm(range(7500)):
    tmp = test[test.fragment_id == i][:60]
    t[i,:,:, 0] = resample(tmp.drop(['fragment_id', 'time_point'],
                                    axis=1), 60, np.array(tmp.time_point))[0]
                  

kfold = StratifiedKFold(5,shuffle=True)


100%|█████████████████████████████████████████████████████████████████████████████| 7500/7500 [00:09<00:00, 754.80it/s]


In [6]:
def Net():
    input = Input(shape=(60, 8, 1))
    X = Conv2D(filters=64,
               kernel_size=(3, 3),
               activation='relu',
               kernel_initializer='he_normal',
               padding='same')(input)
    X = Conv2D(filters=128,
               kernel_size=(3, 3),
               activation='relu',
               kernel_initializer='he_normal',
               padding='same')(X)
    X = MaxPooling2D()(X)
    X = Conv2D(filters=256,
               kernel_size=(3, 3),
               activation='relu',
               kernel_initializer='he_normal',
               padding='same')(X)
    X = Conv2D(filters=512,
               kernel_size=(3, 3),
               activation='relu',
               kernel_initializer='he_normal',
               padding='same')(X)

    X = GlobalMaxPooling2D()(X)
    X = Dropout(0.3)(X)
    X = expand_dims(X,axis=-1)

    
    
    gru_1 = GRU(128,return_sequences=True,kernel_initializer='he_normal', name='gru1')(X)
    gru_1b = GRU(128,return_sequences=True,go_backwards=True,kernel_initializer='he_normal',name='gru1_b')(X)
    gru1_merged = add([gru_1, gru_1b])
    X = Dense(19, kernel_initializer='he_normal',
                  name='dense')(gru1_merged)

    X = Activation('softmax', name='softmax')(X)


    return Model([input], X)

In [52]:
t = t.reshape(7500, 60, 10)
x = x.reshape(7292, 60, 10)
MAX_TIMESTEPS = 60
MAX_NB_VARIABLES = 10
NB_CLASS = 19
x = x.swapaxes(1,2)
t = t.swapaxes(1,2)

In [37]:

proba_t = np.zeros((7500, 19))
proba_val = np.zeros((7292, 19))
for fold, (trn_idx, val_idx) in enumerate(kfold.split(x, y)):
    y_ = to_categorical(y, num_classes=19)
    model = generate_model()
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(),
                  metrics=['acc'])
    plateau = ReduceLROnPlateau(monitor="val_acc",
                                verbose=0,
                                mode='auto',
                                factor=1. / np.cbrt(2),
                                patience=200)
    early_stopping = EarlyStopping(monitor='val_acc',
                                   verbose=0,
                                   mode='auto',
                                   patience=100)
    checkpoint = ModelCheckpoint(f'fold{fold}.h5',
                                 monitor='val_acc',
                                 verbose=0,
                                 mode='auto',
                                 save_best_only=True)
    model.fit(x[trn_idx], y_[trn_idx],
              epochs=500,
              batch_size=512,
              verbose=1,
              shuffle=True,
              validation_data=(x[val_idx], y_[val_idx]),
              callbacks=[plateau, early_stopping, checkpoint])
    model.load_weights(f'fold{fold}.h5')
    proba_t += model.predict(t, verbose=0, batch_size=1024) / 5.
    proba_val[val_idx] = model.predict(x[val_idx]) 

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 10, 60)]     0                                            
__________________________________________________________________________________________________
permute_4 (Permute)             (None, 60, 10)       0           input_9[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 60, 128)      10368       permute_4[0][0]                  
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 60, 128)      512         conv1d_3[0][0]                   
_______________________________________________________________________________________

Epoch 77/500
12/12 [==============================] - 4s 350ms/step - loss: 0.3048 - acc: 0.9016 - val_loss: 1.8339 - val_acc: 0.5613
Epoch 78/500
12/12 [==============================] - 4s 351ms/step - loss: 0.2962 - acc: 0.9073 - val_loss: 1.6905 - val_acc: 0.5305
Epoch 79/500
12/12 [==============================] - 4s 350ms/step - loss: 0.3016 - acc: 0.9023 - val_loss: 1.3743 - val_acc: 0.5942
Epoch 80/500
12/12 [==============================] - 4s 352ms/step - loss: 0.2722 - acc: 0.9103 - val_loss: 1.5200 - val_acc: 0.5565
Epoch 81/500
12/12 [==============================] - 4s 351ms/step - loss: 0.2648 - acc: 0.9131 - val_loss: 1.5540 - val_acc: 0.6093
Epoch 82/500
12/12 [==============================] - 4s 350ms/step - loss: 0.2566 - acc: 0.9158 - val_loss: 1.3904 - val_acc: 0.5942
Epoch 83/500
12/12 [==============================] - 4s 351ms/step - loss: 0.2533 - acc: 0.9179 - val_loss: 1.3574 - val_acc: 0.5977
Epoch 84/500
12/12 [==============================] - 4s 349ms

Epoch 138/500
12/12 [==============================] - 5s 377ms/step - loss: 0.0942 - acc: 0.9765 - val_loss: 1.6336 - val_acc: 0.5990
Epoch 139/500
12/12 [==============================] - 4s 358ms/step - loss: 0.0966 - acc: 0.9755 - val_loss: 1.6854 - val_acc: 0.6189
Epoch 140/500
12/12 [==============================] - 4s 354ms/step - loss: 0.1027 - acc: 0.9707 - val_loss: 1.7624 - val_acc: 0.5744
Epoch 141/500
12/12 [==============================] - 4s 373ms/step - loss: 0.0953 - acc: 0.9757 - val_loss: 1.6209 - val_acc: 0.6004
Epoch 142/500
12/12 [==============================] - 4s 352ms/step - loss: 0.0941 - acc: 0.9753 - val_loss: 1.6766 - val_acc: 0.6107
Epoch 143/500
12/12 [==============================] - 4s 352ms/step - loss: 0.1012 - acc: 0.9721 - val_loss: 1.7519 - val_acc: 0.6134
Epoch 144/500
 8/12 [===================>..........] - ETA: 1s - loss: 0.0955 - acc: 0.9756

KeyboardInterrupt: 

In [89]:
oof_y = np.argmax(y_pred, axis=1)

In [90]:
print(round(accuracy_score(y, oof_y), 5))
score = sum(acc_combo(y_true, y_pred) for y_true, y_pred in zip(y, oof_y)) / oof_y.shape[0]
print(round(score, 5))


0.07913
0.15023


In [61]:
y_pred = model.predict(x) 

In [82]:
y_pred.argmax(1)[:,np.newaxis].shape

(7292, 1)

In [83]:
np.concatenate((mid_layer_output,y_pred.argmax(1)[:,np.newaxis]),axis=1)

array([[ 41.20013046,  23.46347046,  57.5158844 ,   3.        ],
       [ 42.66721344,  29.75338936,  71.60204315,   6.        ],
       [ 36.19981003,  43.88243103,  72.85785675,   6.        ],
       ...,
       [264.80477905, 196.98394775, 317.60546875,   6.        ],
       [194.07733154, 213.60491943, 252.17758179,  10.        ],
       [127.26914978, 111.34897614, 160.05895996,   6.        ]])

In [56]:
from keras.models import load_model
model = load_model('fold4.h5')

from keras import backend as K
mid_layer = Model([model.input],
                       [model.layers[-2].output])
mid_layer_output = mid_layer.predict(x)

In [85]:
hidden_plot = pd.DataFrame(np.concatenate([mid_layer_output,y_pred.argmax(1)[:,np.newaxis]],axis=1),
            columns=['x','y','z','behavior'])

In [60]:
mid_layer_output

array([[ 41.20013 ,  23.46347 ,  57.515884],
       [ 42.667213,  29.75339 ,  71.60204 ],
       [ 36.19981 ,  43.88243 ,  72.85786 ],
       ...,
       [264.80478 , 196.98395 , 317.60547 ],
       [194.07733 , 213.60492 , 252.17758 ],
       [127.26915 , 111.348976, 160.05896 ]], dtype=float32)

In [88]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import math
%matplotlib qt 
#colors = cm.rainbow(np.linspace(0, 1, 19))
colors = [plt.cm.tab10(i/float(19-1)) for i in range(19)]

fig = plt.figure(figsize=(12,12))
ax = Axes3D(fig)
for label,color in zip(range(19),colors):
    cnt = hidden_plot.loc[hidden_plot['behavior']==label].shape[0]
    ax.scatter(hidden_plot.loc[hidden_plot['behavior']==label,'x'],
               hidden_plot.loc[hidden_plot['behavior']==label,'y'],
               hidden_plot.loc[hidden_plot['behavior']==label,'z'],
               color=color,s=cnt/10,alpha=0.5,label=label)

plt.legend()


plt.show()

In [11]:
sub.behavior_id = np.argmax(proba_t,axis=1)
sub.to_csv('sub.csv',index=False)

In [ ]:
def Net():
    input = Input(shape=(60, 8))
 
    X = Conv1D(filters=64,kernel_size=2,padding='same',activation='relu')(input)
    X = BatchNormalization()(X)
    X = MaxPool1D(pool_size=2)(X)
    X = GRU(128,kernel_initializer='he_normal')(input)
    X = Dropout(0.8)(X)
    X = Dense(128,activation='relu')(X)
    X = BatchNormalization()(X)
    X = Dense(256,activation='relu')(X)
    X = Dropout(0.8)(X)
    X = BatchNormalization()(X)
    X = Dense(19,activation='softmax')(X)


    return Model([input], X)

In [ ]:
x = x.reshape(7292,60,8)
t = t.reshape(7500, 60, 8)

In [60]:
def Net1():

    ip = Input(shape=(60,10,1))
    
    x = Reshape(target_shape=(60,10))(ip)
    x = LSTM(16,return_sequences=False)(x)
    #x = AttentionLSTM(16)(x)
    x = Dropout(0.8)(x)
    

    #y = Permute((2, 1))(ip)
    y = Conv2D(filters=128,
               kernel_size=(3, 3),
               activation='relu',
               kernel_initializer='he_uniform',
               padding='same')(ip)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv2D(filters=256,
               kernel_size=(3, 3),
               activation='relu',
               kernel_initializer='he_uniform',
               padding='same')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv2D(filters=128,
               kernel_size=(3, 3),
               activation='relu',
               kernel_initializer='he_uniform',
               padding='same')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling2D()(y)

    x = concatenate([x, y])

    out = Dense(19, activation='softmax')(x)

    model = Model(ip, out)

    model.summary()

    # add load model code here to fine-tune

    return model

In [ ]:
mode = Net1()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 60, 8, 1)]   0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 60, 8, 128)   1280        input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 60, 8, 128)   512         conv2d_15[0][0]                  
__________________________________________________________________________________________________
activation_15 (Activation)      (None, 60, 8, 128)   0           batch_normalization_15[0][0]     
____________________________________________________________________________________________

In [17]:
def generate_model():
    ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))

    x = Masking()(ip)
    x = LSTM(8)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='softmax')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model

In [19]:
tf.config.experimental.list_physical_devices('XLA_GPU')

[PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [8]:
def generate_model_2():
    ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))

    ''' sabsample timesteps to prevent OOM due to Attention LSTM '''
    stride = 2

    x = Permute((2, 1))(ip)
    x = Conv1D(MAX_NB_VARIABLES // stride, 8, strides=stride, padding='same', activation='relu', use_bias=False,
               kernel_initializer='he_uniform')(x) # (None, variables / stride, timesteps)
    x = Permute((2, 1))(x)

    x = Masking()(x)
    x = AttentionLSTM(128)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='softmax')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model